In [ ]:
#@title Install FaceFusion
from IPython.display import clear_output
import torch
import codecs

if torch.cuda.is_available():
  !apt-get update
  !apt-get install -y nvidia-cuda-toolkit
  device="cuda"
  print("Using GPU")
else:
  device="cpu"
  print("Using CPU")


git_repo_rot13 = 'uggcf://tvguho.pbz/snprshfvba/snprshfvba'
git_repo = codecs.decode(git_repo_rot13, 'rot 13')

!git clone $git_repo --branch 2.6.1 --single-branch

directory_rot13 = 'snprshfvba'
directory = codecs.decode(directory_rot13, 'rot 13')

%cd /content/$directory
!python install.py --onnxruntime cuda-12.2 --skip-conda

clear_output()
print("Installed!")

In [ ]:
#@title Run UI

import codecs
import re

%cd /content/$directory

#@markdown The type of tunnel you wanna use for seeing the public link, so if the API of one of them is down, you can use the other one.
Tunnel = "Cloudfare" #@param ["Gradio", "Ngrok", "Cloudfare", "LocalTunnel"]



ngrok_tunnel_authtoken = "" #@param {type:"string"}


if Tunnel == "Gradio":
  file_path_ui_rot13 = "/pbagrag/snprshfvba/snprshfvba/hvf/ynlbhgf/qrsnhyg.cl"
  file_path_ui = codecs.decode(file_path_ui_rot13, 'rot 13')
  !sed -i 's/quiet = True/quiet=True,share=True/g' $file_path_ui
elif Tunnel == "Ngrok":
  !sed -i 's/quiet=True,share=True/quiet = True/g' $file_path_ui
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  http_tunnel = ngrok.connect(7860, bind_tls=True)
  clear_output()
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudfare":
  !sed -i 's/quiet=True,share=True/quiet = True/g' $file_path_ui
  # download cloudfare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:7860 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudfare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
  !sed -i 's/quiet=True,share=True/quiet = True/g' $file_path_ui
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()
  print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnel Password: {endpoint_ip}')


if device=="cuda":
  !python run.py --execution-providers cuda
else:
  !python run.py --execution-providers cpu